# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Autumn 2025** </center>
---
### <center> **Examples on Structured Streaming (Kakfa producer)** </center>
---
**Profesor**: Pablo Camarillo Ramirez

# Create SparkSession

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming (Kafka)") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.0") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

# Create a data stream from a Kafka topic

In [ ]:
# Create the remote connection
kafka_df = spark.readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "kafka:9093") \
            .option("subscribe", " videogame-telemetry-1") \
            .load()

kafka_df.printSchema()

In [ ]:
from pyspark.sql.functions import year, month, day, from_json, col
from pyspark.sql.types import StructField, StringType
from pcamarillor.spark_utils import SparkUtils
vg_telemetry_df = kafka_df.select(kafka_df.value.cast("string").alias("value_str"))

# We need to extract the columns from the input JSON
location_schema = SparkUtils.generate_schema([("latitude", "float"), ("longitude", "float")])
schema_columns = [("player_id", "string"),
                  ("timestamp", "string"),
                  ("device", "string"),
                  ("action", "string"),
                  ("speed", "float"),
                  ("battery_level", "float")]
pkg_schema = SparkUtils.generate_schema(schema_columns)
pkg_schema = pkg_schema.add(StructField("location", location_schema))
                                       

In [ ]:
# Extract the year, monht, and day from the timestamp column


In [ ]:
!mkdir data/vg_output

In [ ]:
!pwd
!ls data/vg_output/

In [ ]:
# Finally, we send the stream to a files sink
query_files = vg_extracted_time_df.writeStream \
                .trigger(processingTime="10 seconds") \
                .partitionBy("year", "month", "day") \
                .format("parquet") \
                .option("header", "true") \
                .option("path", "/opt/spark/work-dir/data/vg_output/") \
                .option("checkpointLocation", "/opt/spark/work-dir/vg_checkpoint") \
                .start()

In [ ]:
!ls /opt/spark/work-dir/data/vg_output/year=2025/month=10

In [ ]:
sc.stop()